In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [2]:
def get_uvw(arr='a',decl=34.0784,freq=1.4e9):
    if arr=='a':
        array=np.loadtxt("./vla_a_array.txt")
    else:
        array=np.loadtxt("./vla_d_array.txt")
    xyz=array[:,:3]*3e8*1e-9
    N=xyz.shape[0]
    
    print(f"Frequency is {freq/1e9:4.2f} GHz")

    #take phi=0 at the longitude of the array. little bit arbitrary, could as well have fixed East as Greenwich.
    #Will have to subract (or add?) the longitude of the array from phi to get the correct hour angle.
    
    decl=np.deg2rad(decl)
    east=[0,1,0]
    zen = [np.cos(decl),0,np.sin(decl)]
    north=np.cross(zen,east)

    proj_mat = np.vstack([east,north,zen]).T

    nb = N*(N-1)//2
    
#     print("Num baselines = ", nb)

    uvw=np.zeros((nb,3))
    
#     print(f"Vertical scatter in array {arr} is {np.std(xyz[:2]):4.2f} m")

    count=0
    for i in range(N):
        for j in range(i+1,N):
            uvw[count,:] = xyz[i,:]-xyz[j,:] #Baselines in m, in Earth Center ref frame. will project later
            count+=1
#     print(f"Max baseline for array {arr} is {np.max(np.sqrt(np.sum(uvw**2,axis=1))):4.2f} m")
    uvw=np.vstack([uvw,-uvw])*freq/3e8 # since image is real, it's FT needs to bbe hermitian conjugate F*(-k) = F(k) = 1
    return xyz,uvw,proj_mat

In [9]:
rms = lambda x: np.sqrt(np.mean(x**2))


In [47]:
# UVW frame centered at source with declination decl

array='d'
freq=1.42e9
xyz,uvw,proj_mat=get_uvw(arr=array,freq=freq) #source at zenith

# Dishes are 25m wide
FWHM = 3e8/1.4e9/25
FWHM=np.rad2deg(FWHM)

print("FWHM in arcsec is:", FWHM*3600)

decl=34.0784
#source overhead
# mydecl=(decl)*np.pi/180
# source1_dir = [np.cos(mydecl),0,np.sin(mydecl)]
mydecl=(decl-FWHM)*np.pi/180 #Second source 1 FWHM to south
source2_dir = [np.cos(mydecl),0,np.sin(mydecl)]

# n1=source1_dir@proj_mat   #I will work in a rotated coordinate frame (centered on the source)
n2=source2_dir@proj_mat

uvw_source1 = uvw.copy() 
uvw_source1[:] = uvw_source1@proj_mat  # baselines in source1 UVW coordinate frame
uvw_flat_source1 = uvw_source1.copy()
uvw_flat_source1[:,2]=0 #0 out the w term to make all baselines 2D in source1 UVW plane
#path length difference of sources to the baselines. 

Frequency is 1.40 GHz
FWHM in arcsec is: 1767.9840535465398


In [48]:
pathdiff = uvw_flat_source1@n2

In [46]:
print(2*np.pi*rms(pathdiff)) # at 1.42 GHz phase has changed by 1 rad

76.740680736981


In [49]:
print(2*np.pi*rms(pathdiff)) # phase at 1.4 GHz 

75.65982607871368


### D array has short baselines, and thus can tolerate a large span of bandwidth. Need channel resolution $<= 200 MHz$.

In [65]:
# UVW frame centered at source with declination decl

array='a'
freq=1.4006e9
xyz,uvw,proj_mat=get_uvw(arr=array,freq=freq) #source at zenith

# Dishes are 25m wide
FWHM = 3e8/1.4e9/25
FWHM=np.rad2deg(FWHM)

print("FWHM in arcsec is:", FWHM*3600)

decl=34.0784
#source overhead
# mydecl=(decl)*np.pi/180
# source1_dir = [np.cos(mydecl),0,np.sin(mydecl)]
mydecl=(decl-FWHM)*np.pi/180 #Second source 1 FWHM to south
source2_dir = [np.cos(mydecl),0,np.sin(mydecl)]

# n1=source1_dir@proj_mat   #I will work in a rotated coordinate frame (centered on the source)
n2=source2_dir@proj_mat

uvw_source1 = uvw.copy() 
uvw_source1[:] = uvw_source1@proj_mat  # baselines in source1 UVW coordinate frame
uvw_flat_source1 = uvw_source1.copy()
uvw_flat_source1[:,2]=0 #0 out the w term to make all baselines 2D in source1 UVW plane
#path length difference of sources to the baselines. 

Frequency is 1.40 GHz
FWHM in arcsec is: 1767.9840535465398


In [66]:
pathdiff = uvw_flat_source1@n2

In [67]:
print(2*np.pi*rms(pathdiff)) # at 1.4006 GHz phase has changed by 1 rad

2669.8624930238393


In [52]:
print(2*np.pi*rms(pathdiff)) # phase at 1.4 GHz 

2668.7187564139476


### A array has larger baselines, and thus smearing occurs in short span of bandwidth. Need channel resolution $<= 6 MHz$.